In [85]:
import pandas as pd
import numpy as np
import jellyfish
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import time

In [86]:
raw = pd.read_csv("test.csv", index_col=[0])

In [87]:
raw.columns

Index(['business_name', 'business_address', 'business_city', 'business_state',
       'business_postal_code', 'business_latitude', 'business_longitude',
       'business_phone_number', 'inspection_id', 'inspection_date',
       'inspection_score', 'inspection_type', 'violation_id',
       'violation_description', 'risk_category'],
      dtype='object')

In [94]:
def token_set(name, value):
    output = fuzz.token_set_ratio(name, value)
    return output

def token_sort(name, value):
    output = fuzz.token_sort_ratio(name, value)
    return output

def clean_dirty_inspection(df):
    set_sort = np.vectorize(token_set)
    sort_sort = np.vectorize(token_sort)
    start = time.time()
    idct = {}
    iterator =0
    df = df[['business_name', 'business_address', 'business_postal_code']]
    for zipcode, rows in df.groupby("business_postal_code"):
        name_list = list(rows['business_name'])
        addr_list = list(rows['business_address'])
        tuple_list = set(zip(name_list, addr_list))
        for t in tuple_list:
            name, address = t
            if name not in rows['business_name']:
                pass
            rows.loc[:, 'name_sim'] = set_sort(rows['business_name'], name)
            rows.loc[:, 'addr_sim'] = sort_sort(rows['business_address'], address)

            same = rows.loc[(rows['name_sim'] >= 80)]
            same = same.loc[(same['addr_sim'] >= 80)]
            ix = same.index.get_level_values(0)
            x=list(ix)
            if x != []:
                idct[iterator] = x
                iterator += 1
            rows = rows[~rows.isin(same).dropna()]
            df = df[~df.index.isin(same.index)]
    end = time.time()
    print(end-start)
    return idct

In [88]:
raw=raw.dropna(subset=['business_name', 'business_address'])

In [89]:
raw.shape

(50630, 15)

In [90]:
raw

,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
0,Dar Bar Pakistani/Indian Cusine,1412 Polk St,San Francisco,CA,94109,37.789784,-122.420455,NaN,1757_20170928,2017-09-28,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk
1,DRAGON CITY BAKERY & CAFE,2367 MISSION St,San Francisco,CA,94110,37.759174,-122.419066,1.415583e+10,4864_20161206,2017-12-06,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk
2,Deli 23,2449 23rd St,San Francisco,CA,94110,NaN,NaN,NaN,79782_20160503,2017-05-03,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk
3,L'acajou Bakery and Cafe,498 09th St Ste. C,San Francisco,CA,94103,NaN,NaN,NaN,73840_20171207,2017-12-07,71.0,Routine - Unscheduled,73840_20171207_103105,Improper cooling methods,High Risk
4,Sweetheart Cafe,909 Grant Ave,San Francisco,CA,94108,NaN,NaN,NaN,76437_20160329,2017-03-29,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk
5,Cafe New Honolulu,888 Stockton St,San Francisco,CA,94108,NaN,NaN,NaN,82123_20171207,2017-12-07,87.0,Routine - Unscheduled,82123_20171207_103131,Moderate risk vermin infestation,Moderate Risk
6,Aux Delices Vietnam,2327 Polk St,San Francisco,CA,94109,37.798274,-122.422264,NaN,1146_20180223,2017-02-23,77.0,Routine - Unscheduled,1146_20180223_103103,High risk food holding temperature,High Risk
7,CAFE PICARO,3120 16th St,San Francisco,CA,94103,37.764908,-122.422442,1.415543e+10,3838_20180222,2017-02-22,72.0,Routine - Unscheduled,3838_20180222_103142,Unclean nonfood contact surfaces,Low Risk
8,Jelly Donut,3198 24th St,San Francisco,CA,94110,37.752439,-122.416250,NaN,38929_20160114,2017-01-14,88.0,Routine - Unscheduled,38929_20160114_103139,Improper food storage,Low Risk
9,Limon Peruvian Rotisserie,1001 South Van Ness Ave,San Francisco,CA,94110,NaN,NaN,1.415555e+10,81161_20160325,2017-03-25,92.0,Routine - Unscheduled,81161_20160325_103149,Wiping cloths not clean or properly stored or ...,Low Risk


In [102]:
ins, res = link_record(raw, idc)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


551.2213959693909


In [97]:
idc = clean_dirty_inspection(raw)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


1772.460058927536


In [104]:
res

,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category,business_id
0,1757_20170928,2017-09-28,86.0,Routine - Unscheduled,1757_20170928_103131,Moderate risk vermin infestation,Moderate Risk,2024.0
1,4864_20161206,2017-12-06,84.0,Routine - Unscheduled,4864_20161206_103157,Food safety certificate or food handler card n...,Low Risk,2567.0
2,79782_20160503,2017-05-03,92.0,Routine - Unscheduled,79782_20160503_103120,Moderate risk food holding temperature,Moderate Risk,2758.0
3,73840_20171207,2017-12-07,71.0,Routine - Unscheduled,73840_20171207_103105,Improper cooling methods,High Risk,670.0
4,76437_20160329,2017-03-29,76.0,Routine - Unscheduled,76437_20160329_103113,Sewage or wastewater contamination,High Risk,1740.0
5,82123_20171207,2017-12-07,87.0,Routine - Unscheduled,82123_20171207_103131,Moderate risk vermin infestation,Moderate Risk,1776.0
6,1146_20180223,2017-02-23,77.0,Routine - Unscheduled,1146_20180223_103103,High risk food holding temperature,High Risk,2149.0
7,3838_20180222,2017-02-22,72.0,Routine - Unscheduled,3838_20180222_103142,Unclean nonfood contact surfaces,Low Risk,597.0
8,38929_20160114,2017-01-14,88.0,Routine - Unscheduled,38929_20160114_103139,Improper food storage,Low Risk,2349.0
9,81161_20160325,2017-03-25,92.0,Routine - Unscheduled,81161_20160325_103149,Wiping cloths not clean or properly stored or ...,Low Risk,2298.0


In [101]:
len(idc)

5662

In [100]:
def link_record(df, idct):
    '''
    Using the dicitionary created, matching the unique resturant's business_id to index in both
    rest_df and insp_df
    '''
    start = time.time()
    insp_columns = ['inspection_id', 'inspection_date', 'inspection_score', 'inspection_type', 'violation_id', 'violation_description', 'risk_category']
    insp_df = df[insp_columns]
    rest_columns = ['business_name', 'business_address', 'business_city','business_state', 'business_postal_code','business_latitude', 'business_longitude', 'business_phone_number']                    
    rest_df = df[rest_columns]
    
    for k, v in idct.items():
        insp_df.loc[v, "business_id"]= int(k)
        rest_df.loc[v[-1], 'business_id']= int(k)
    rest_df = rest_df.dropna(subset=["business_id"])
    end = time.time()
    print(end-start)
    return rest_df, insp_df
    

In [69]:
mni

,,business_name,business_address,business_city,business_state,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
,business_postal_code,,,,,,,,,,,,,,
50610,94109,Miller's East Coast Deli Restaurant,1725 Polk St,San Francisco,CA,37.792784,-122.421164,NaN,33442_20160216,2017-02-16,75.0,Routine - Unscheduled,33442_20160216_103103,High risk food holding temperature,High Risk
50611,94109,Millers East Coast Deli,1725 Polk St,San Francisco,CA,37.792784,-122.421164,NaN,33442_20170807,2017-08-07,82.0,Routine - Unscheduled,33442_20170807_103103,High risk food holding temperature,High Risk
50612,94123,Umamu Birger SF,2184 Union St,San Francisco,CA,37.797168,-122.435124,NaN,67357_20151030,2017-10-30,NaN,Reinspection/Followup,NaN,NaN,NaN
50613,94123,Umami Burger SF,2184 tnion SU,San Francisco,CA,37.797168,-122.435124,NaN,67357_20171019,2017-10-19,87.0,Routine - Unscheduled,67357_20171019_103156,Permit license or inspection report not posted,Low Risk
50614,94123,Umami Burger SF,8124 Union St,San Francisco,CA,37.797168,-122.435124,NaN,67357_20170524,2017-05-24,86.0,Routine - Unscheduled,67357_20170524_103157,Food safety certificate or food handler card n...,Low Risk
50615,94123,Burger Umami SF,2184 Union St,San Francisco,CA,37.797168,-122.435124,NaN,67357_20151007,2017-10-07,90.0,Routine - Unscheduled,67357_20151007_103159,Worker safety hazards,Low Risk
50616,94121,Jade Garden,3608 BALBOA Street,San Francisco,CA,37.775728,-122.497612,NaN,61161_20170420,2017-04-20,78.0,Routine - Unscheduled,61161_20170420_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk
50617,94121,Jade Garden,3068 BALBOA St,San Francisco,CA,37.775728,-122.497612,NaN,61161_20160107,2017-01-07,59.0,Routine - Unscheduled,61161_20160107_103142,Unclean nonfood contact surfaces,Low Risk
50618,94121,Jade Garden Restaurant,3608 BALBOA St,San Francisco,CA,37.775728,-122.497612,NaN,61161_20170420,2017-04-20,78.0,Routine - Unscheduled,61161_20170420_103120,Moderate risk food holding temperature,Moderate Risk


In [68]:
# a 10-row mini test df
mni =raw.iloc[50610:50619]

In [24]:
mni

,Unnamed: 0,business_name,business_address,business_city,business_state,business_latitude,business_longitude,business_phone_number,inspection_id,inspection_date,inspection_score,inspection_type,violation_id,violation_description,risk_category
business_postal_code,,,,,,,,,,,,,,,
94109,50610,Miller's East Coast Deli Restaurant,1725 Polk St,San Francisco,CA,37.792784,-122.421164,NaN,33442_20160216,2017-02-16,75.0,Routine - Unscheduled,33442_20160216_103103,High risk food holding temperature,High Risk
94109,50611,Millers East Coast Deli,1725 Polk St,San Francisco,CA,37.792784,-122.421164,NaN,33442_20170807,2017-08-07,82.0,Routine - Unscheduled,33442_20170807_103103,High risk food holding temperature,High Risk
94123,50612,Umamu Birger SF,2184 Union St,San Francisco,CA,37.797168,-122.435124,NaN,67357_20151030,2017-10-30,NaN,Reinspection/Followup,NaN,NaN,NaN
94123,50613,Umami Burger SF,2184 tnion SU,San Francisco,CA,37.797168,-122.435124,NaN,67357_20171019,2017-10-19,87.0,Routine - Unscheduled,67357_20171019_103156,Permit license or inspection report not posted,Low Risk
94123,50614,Umami Burger SF,8124 Union St,San Francisco,CA,37.797168,-122.435124,NaN,67357_20170524,2017-05-24,86.0,Routine - Unscheduled,67357_20170524_103157,Food safety certificate or food handler card n...,Low Risk
94123,50615,Burger Umami SF,2184 Union St,San Francisco,CA,37.797168,-122.435124,NaN,67357_20151007,2017-10-07,90.0,Routine - Unscheduled,67357_20151007_103159,Worker safety hazards,Low Risk
94121,50616,Jade Garden,3608 BALBOA Street,San Francisco,CA,37.775728,-122.497612,NaN,61161_20170420,2017-04-20,78.0,Routine - Unscheduled,61161_20170420_103124,Inadequately cleaned or sanitized food contact...,Moderate Risk
94121,50617,Jade Garden,3068 BALBOA St,San Francisco,CA,37.775728,-122.497612,NaN,61161_20160107,2017-01-07,59.0,Routine - Unscheduled,61161_20160107_103142,Unclean nonfood contact surfaces,Low Risk
94121,50618,Jade Garden Restaurant,3608 BALBOA St,San Francisco,CA,37.775728,-122.497612,NaN,61161_20170420,2017-04-20,78.0,Routine - Unscheduled,61161_20170420_103120,Moderate risk food holding temperature,Moderate Risk


In [55]:
res =link_record(df, idc)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


189.76967406272888


In [68]:
#clean_dirty_inspection(raw[:1000])

In [ ]:
ins

In [ ]:
res

In [65]:
t2.head()

,business_postal_code,tuple
0,94109,"(Dar Bar Pakistani/Indian Cusine, 1412 Polk St)"
1,94110,"(DRAGON CITY BAKERY & CAFE, 2367 MISSION St)"
2,94110,"(Deli 23, 2449 23rd St)"
3,94103,"(L'acajou Bakery and Cafe, 498 09th St Ste. C)"
4,94108,"(Sweetheart Cafe, 909 Grant Ave)"


In [63]:
def get_2str(df):
    '''
    Inputs:
        a pandas dataframe

    Returns: a list of tuples contains the three column values of each row 
            converted to a pandas Series
    '''
    str_lst =[]
    for i, row in df.iterrows():
        str_lst.append((row.business_name, row.business_address))
    rv = pd.Series(str_lst)
    rv = pd.concat([df["business_postal_code"], rv], axis=1)
    rv.columns = ["business_postal_code",'tuple']
    return rv


In [ ]:
def get_tuple(a, b):
    '''
    Inputs:
        one row from each of the two parallel dataframe

    Returns: a single tuple contains three categories
    '''
    j = jellyfish.jaro_winkler(a, b)
    name_score = get_jw_category(j)

    return name_score

In [ ]:
THRESH1 = 0.8
THRESH2 = 1.0

def get_jw_category(j):
    '''
    Convert a Jaro-Winkler score into a categorical: low, medium. high
    using fixed thresholds.

    Inputs:
        j (double): value between 0 and 1 (inclusive)

    Returns: string
    '''
    if j < THRESH1:
        return "low"
    if j < THRESH2:
        return "medium"
    return "high"